## Prompt Response Relevance Feedback Requirements
1. Relevance requires adherence to the entire prompt.
2. Responses that don't provide a definitive answer can still be relevant
3. Admitting lack of knowledge and refusals are still relevant.
4. Feedback mechanism should differentiate between seeming and actual relevance.
5. Relevant but inconclusive statements should get increasingly high scores as they are more helpful for answering the query.

Below are examples of usage. Find the [results](#test-results) of the tests tabulated at the bottom.

In [ ]:
# Import relevance feedback function
from trulens_eval.feedback import GroundTruthAgreement, OpenAI as fOpenAI
from trulens_eval import TruBasicApp, Feedback, Tru, Select
from test_cases import answer_relevance_golden_set


import openai

Tru().reset_database()

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "..."

In [3]:
turbo = fOpenAI(model_engine="gpt-3.5-turbo")
# Define your feedback functions
def wrapped_relevance_turbo(input, output):
    return turbo.relevance(input, output)

# Define your feedback functions
def wrapped_relevance_with_cot_turbo(input, output):
    return turbo.relevance_with_cot_reasons(input, output)

gpt4 = fOpenAI(model_engine="gpt-4")
# Define your feedback functions
def wrapped_relevance_gpt4(input, output):
    return gpt4.relevance(input, output)

# Define your feedback functions
def wrapped_relevance_with_cot_gpt4(input, output):
    return gpt4.relevance_with_cot_reasons(input, output)

In [4]:
# Create a Feedback object using the numeric_difference method of the ground_truth object
ground_truth = GroundTruthAgreement(answer_relevance_golden_set)
# Call the numeric_difference method with app and record
f_groundtruth = Feedback(ground_truth.numeric_difference, name = "Relevance Smoke Test").on(Select.Record.calls[0].args.args[0]).on(Select.Record.calls[0].args.args[1]).on_output()

✅ In Relevance Smoke Test, input prompt will be set to *.__record__.calls[0].args.args[0] .
✅ In Relevance Smoke Test, input response will be set to *.__record__.calls[0].args.args[1] .
✅ In Relevance Smoke Test, input score will be set to *.__record__.main_output or `Select.RecordOutput` .


In [5]:
tru_wrapped_relevance_turbo = TruBasicApp(wrapped_relevance_turbo, app_id = "answer relevance gpt-3.5-turbo", feedbacks=[f_groundtruth])
tru_wrapped_relevance_with_cot_turbo = TruBasicApp(wrapped_relevance_with_cot_turbo, app_id = "answer relevance with cot reasoning gpt-3.5-turbo", feedbacks=[f_groundtruth])
tru_wrapped_relevance_gpt4 = TruBasicApp(wrapped_relevance_gpt4, app_id = "answer relevance gpt-4", feedbacks=[f_groundtruth])
tru_wrapped_relevance_with_cot_gpt4 = TruBasicApp(wrapped_relevance_with_cot_gpt4, app_id = "answer relevance with cot reasoning gpt-4", feedbacks=[f_groundtruth])

In [6]:
for i in range(len(answer_relevance_golden_set)):
    prompt = answer_relevance_golden_set[i]["query"]
    response = answer_relevance_golden_set[i]["response"]
    tru_wrapped_relevance_turbo.call_with_record(prompt, response)
    tru_wrapped_relevance_with_cot_turbo.call_with_record(prompt, response)
    tru_wrapped_relevance_gpt4.call_with_record(prompt, response)
    tru_wrapped_relevance_with_cot_gpt4.call_with_record(prompt, response)

Task queue full. Finishing existing tasks.
Task queue full. Finishing existing tasks.


In [8]:
Tru().get_records_and_feedback(app_ids=[])[0].groupby('app_id')[['Relevance Smoke Test', 'latency', 'total_cost']].mean()

,Relevance Smoke Test,latency,total_cost
app_id,,,
answer relevance gpt-3.5-turbo,0.794118,0.058824,0.000763
answer relevance gpt-4,0.764706,0.058824,0.015277
answer relevance with cot reasoning gpt-3.5-turbo,0.770588,0.058824,0.000912
answer relevance with cot reasoning gpt-4,0.764706,0.058824,0.019181
